## Análise do dataset

Neste notebook iremos apenas analisar como os dados estão distribuidos, com visualizações e verificações de inconsistências, além de fazer sugestões para lidar com os diferentes tipos de dados, e também como iremos enfrentar os dados faltantes.

Assim, aqui não será realizado nenhuma técnica de pré-processamento de fato, isto será feito no próximo notebook, este notebook tem o único intuito de analisar os atributos, verificar seus tipos e valores.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Lendo o conjunto de dados

df = pd.read_csv("test.csv")

#Exibindo o cabeçalho do conjunto de dados
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [3]:
# Tamanho do conjunto de dados
df.shape

(1459, 80)

Podemos compreender que termos muitos atributos para poder lidar durante a nossa análise exploratória, e como podemos ver existem diversos valores NULOS que devem ser tratados, além obviamente de tratar os atributos categóricos, que devem ser transformados para numérico, mas isto deve ser feito com cuidado, já que caso usemos One-Hot-Encoding, podemos ter um número de atributos preditivos muito grande, o que pode atrapalhar o modelo de regressão.

In [4]:
# Verificando os tipos de atributo
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

Temos 38 atributos numéricos e 43 atributos categóricos. Logo teremos bastante trabalho pela frente. Além disso podemos perceber que existem diversos valores nulos

In [5]:
df.isna().sum().sort_values(ascending=False)[:20]

PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
FireplaceQu      730
LotFrontage      227
GarageYrBlt       78
GarageQual        78
GarageFinish      78
GarageCond        78
GarageType        76
BsmtCond          45
BsmtQual          44
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
MasVnrType        16
MasVnrArea        15
MSZoning           4
BsmtHalfBath       2
dtype: int64

Podemos perceber acima, que , como temos 1460 linhas, existem alguns atributos que contém a maioria dos valores NULOS, o que é muito problemático. Assim, é extremamente necessário tratar valores NULOS

In [6]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

## Pré-Processamento

Abaixo iremos tratar os valores NULOS e as incoêrencias do dataset

### Tratando valores NULOS e Categóricos ORDINAIS

In [7]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [8]:
# Remover o atributo ID

df.drop('Id',axis=1,inplace=True)

In [9]:
# Mapear atributo "Street" para binário
df['Street'] =  df['Street'].map({'Pave': 1, 'Grvl': 0})

In [10]:
# Remover o atributo Alley
df.drop('Alley',axis=1,inplace=True)

In [11]:
# Utilizar lable encoder (Categórico Ordinal)
df['LotShape'] =  df['LotShape'].map({'Reg': 3, 'IR1':2, 'IR2':1, 'IR3':0 })

In [43]:
# Utilizar label encoder (Categórico Ordinal)
df['Utilities'] =  df['Utilities'].map({'AllPub': 3, 'NoSewr':2, 'NoSeWa':1, 'ELO':0 })
df.Utilities.fillna(value=0,inplace=True)


In [13]:
# Utilizar label encoder (Categórico Ordinal)
df['LandSlope'] =  df['LandSlope'].map({'Gtl': 2, 'Mod':1, 'Sev':0})

In [14]:
# Utilizar label encoder (Categórico Ordinal)
df['ExterQual'] =  df['ExterQual'].map({'Ex': 4, 'Gd':3, 'TA':2, 'Fa':1,'Po':0})

In [15]:
# Utilizar label encoder (Categórico Ordinal)
df['ExterCond'] =  df['ExterCond'].map({'Ex': 4, 'Gd':3, 'TA':2, 'Fa':1,'Po':0})

In [16]:
# Utilizar label encoder (Categórico Ordinal)
df['BsmtQual'] =  df['BsmtQual'].map({'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1})
df.BsmtQual.fillna(value=0,inplace=True)
df['BsmtQual'].isna().sum()

0

In [17]:
# Utilizar label encoder (Categórico Ordinal)
df['BsmtCond'] =  df['BsmtCond'].map({'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1})
df.BsmtCond.fillna(value=0,inplace=True)
df['BsmtCond'].isna().sum()

0

In [18]:
# Utilizar label encoder (Categórico Ordinal)
df['BsmtExposure'] =  df['BsmtExposure'].map({'Gd': 4, 'Av':3, 'Mn':2, 'No':1})
df.BsmtExposure.fillna(value=0,inplace=True)
df['BsmtExposure'].isna().sum()

0

In [19]:
# Utilizar label encoder (Categórico Ordinal)
df['BsmtFinType1'] =  df['BsmtFinType1'].map({'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1})
df.BsmtFinType1.fillna(value=0,inplace=True)
df['BsmtFinType1'].isna().sum()

0

In [20]:
# Utilizar label encoder (Categórico Ordinal)
df['BsmtFinType2'] =  df['BsmtFinType2'].map({'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1})
df.BsmtFinType2.fillna(value=0,inplace=True)
df['BsmtFinType2'].isna().sum()

0

In [21]:
# Utilizar label encoder (Categórico Ordinal)
df['HeatingQC'] =  df['HeatingQC'].map({'Ex': 4, 'Gd':3, 'TA':2, 'Fa':1,'Po':0})

In [22]:
# Utilizar label encoder (Categórico Ordinal)
df['CentralAir'] =  df['CentralAir'].map({'Y':1,'N':0})

In [23]:
# Utilizar label encoder (Categórico Ordinal)
df['KitchenQual'] =  df['KitchenQual'].map({'Ex': 4, 'Gd':3, 'TA':2, 'Fa':1,'Po':0})

In [24]:
df['Functional'].unique()

array(['Typ', 'Min2', 'Min1', 'Mod', 'Maj1', 'Sev', 'Maj2', nan],
      dtype=object)

In [25]:
# Utilizar label encoder (Categórico Ordinal)
df['Functional'] =  df['Functional'].map({'Typ':7, 'Min1':6, 'Min2':5,'Mod':4,  'Maj1':3,'Maj2':2
                                          , 'Sev':1, 'Sal':0})

In [26]:
# Utilizar label encoder (Categórico Ordinal)
df['FireplaceQu'] =  df['FireplaceQu'].map({'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1})
df.FireplaceQu.fillna(value=0,inplace=True)
df['FireplaceQu'].isna().sum()

0

In [27]:
# Utilizar label encoder (Categórico Ordinal)
df['GarageFinish'] =  df['GarageFinish'].map({'Fin':3,'RFn':2,'Unf':1})
df.GarageFinish.fillna(value=0,inplace=True)
df['GarageFinish'].isna().sum()

0

In [28]:
# Utilizar label encoder (Categórico Ordinal)
df['GarageQual'] =  df['GarageQual'].map({'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1})
df.GarageQual.fillna(value=0,inplace=True)
df['GarageQual'].isna().sum()

0

In [29]:
# Utilizar label encoder (Categórico Ordinal)
df['GarageCond'] =  df['GarageCond'].map({'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2,'Po':1})
df.GarageCond.fillna(value=0,inplace=True)
df['GarageCond'].isna().sum()

0

In [30]:
# Remover o atributo PoolQC

df.drop('PoolQC',axis=1,inplace=True)

In [31]:
# Remover o atributo Fence

df.drop('Fence',axis=1,inplace=True)

In [34]:
# Corrigindo o atributo LotFrontage. Atribuindo a média aos valores NULOS
df.LotFrontage.fillna(value=df.LotFrontage.mean(),inplace=True)
df.LotFrontage.isna().sum()

0

In [35]:
# Corrigindo o atributo GarageYrBlt. Temos 81 valores NULOS. Atrelar a data de construção do Prédio.

df.GarageYrBlt.fillna(value=df.YearBuilt ,inplace=True)

In [36]:
# None é um valor válido para este atributo
df.MasVnrType.fillna(value='No',inplace=True)
df['MasVnrType'].isna().sum()

0

In [37]:
# None é um valor válido para este atributo
df.MasVnrArea.fillna(value=0,inplace=True)
df['MasVnrArea'].isna().sum()

0

In [38]:
def put_mode(col_name,col_ref,df):

    indexes = (np.where(df[col_name].isna()))

    modes = {}
    
    for i in indexes[0]:

        value = df.loc[i,col_ref]

        modes[i] = df[df[col_ref] == value][col_name].mode().values[0]

    df[col_name].fillna(value=modes,inplace=True)
    
    

In [39]:
put_mode('Electrical','Utilities',df)
df['Electrical'].isna().sum()

0

In [40]:
# None é um valor válido para este atributo
df.GarageType.fillna(value='No',inplace=True)
df['GarageType'].isna().sum()

0

In [41]:
# None é um valor válido para este atributo
df.MiscFeature.fillna(value='No',inplace=True)
df['MiscFeature'].isna().sum()

0

Os seguintes atributos precisam ser tratados.


 1   MSZoning       1455 non-null   object 
 
 
 21  Exterior1st    1458 non-null   object 
 
 22  Exterior2nd    1458 non-null   object 

 32  BsmtFinSF1     1458 non-null   float64
 
 34  BsmtFinSF2     1458 non-null   float64
 
 35  BsmtUnfSF      1458 non-null   float64
 
 36  TotalBsmtSF    1458 non-null   float64
 
 45  BsmtFullBath   1457 non-null   float64
 
 46  BsmtHalfBath   1457 non-null   float64

 51  KitchenQual    1458 non-null   float64
 
 53  Functional     1457 non-null   float64

 59  GarageCars     1458 non-null   float64
 
 60  GarageArea     1458 non-null   float64
  
 74  SaleType       1458 non-null   object 

In [46]:
# Tratando valor missing
df.BsmtFinSF1.fillna(value=0,inplace=True)
df['BsmtFinSF1'].isna().sum()

0

In [47]:
# Tratando valor missing
df.BsmtFinSF2.fillna(value=0,inplace=True)
df['BsmtFinSF2'].isna().sum()

0

In [48]:
# Tratando valor missing
df.BsmtUnfSF.fillna(value=0,inplace=True)
df['BsmtUnfSF'].isna().sum()

0

In [49]:
# Tratando valor missing
df.TotalBsmtSF.fillna(value=0,inplace=True)
df['TotalBsmtSF'].isna().sum()

0

In [50]:
# Tratando valor missing
df.BsmtFullBath.fillna(value=0,inplace=True)
df['BsmtFullBath'].isna().sum()

0

In [51]:
# Tratando valor missing
df.BsmtHalfBath.fillna(value=0,inplace=True)
df['BsmtHalfBath'].isna().sum()

0

In [53]:
# Tratando valor missing
df.KitchenQual.fillna(value=0,inplace=True)
df['KitchenQual'].isna().sum()

0

In [52]:
# Tratando valor missing
df.Functional.fillna(value=0,inplace=True)
df['Functional'].isna().sum()

0

In [54]:
# Tratando valor missing
df.GarageCars.fillna(value=0,inplace=True)
df['GarageCars'].isna().sum()

0

In [55]:
# Tratando valor missing
df.GarageArea.fillna(value=0,inplace=True)
df['GarageArea'].isna().sum()

0

Após tratarmos os floats, iremos lidar com as strings.


 1   MSZoning       1455 non-null   object 

 21  Exterior1st    1458 non-null   object 
 
 74  SaleType       1458 non-null   object 


In [65]:
put_mode('MSZoning','MSSubClass',df)
df['MSZoning'].isna().sum()

0

In [64]:
put_mode('Exterior1st','Condition1',df)
df['Exterior1st'].isna().sum()

0

In [72]:
put_mode('Exterior2nd','Condition1',df)
df['Exterior2nd'].isna().sum()

0

In [63]:
put_mode('SaleType','SaleCondition',df)
df['SaleType'].isna().sum()

0

In [73]:
df.isna().sum().sum()

0

### Encodings

Para a tarefa de inferência, seremos capazes de realizar o target encoder. Entretanto, para predição, que não tem a classe, seremos obrigados a utilizar outro tipo de encoder.

Iremos utilizar 1 abordagem para a predição:

- Remover atributos categóricos restantes (24)


### Verificando atributos ambíguos

Este atributos tem uma correlação alta, o que fazer?

Iremos testar inicialmente a regressão com estes atributos, e depois removê-los, e comparar os modelos. E comparar o R2 ajustado dos atributos para verificar se houve melhora.

In [87]:
remover = ['GarageCond','SaleType','Exterior2nd']

df.drop(remover,axis=1,inplace=True)

KeyError: "['GarageCond', 'SaleType', 'Exterior2nd'] not found in axis"

In [81]:
values = [
    "MSZoning",
    "LandContour",
    "LotConfig",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Foundation",
    "PavedDrive",
    "SaleCondition",
    "Heating","MasVnrType",
    "Electrical",
    "GarageType",
    'MiscFeature'
]

In [88]:
df_num = df.copy()

# Removendo todos os atributos categóricos
df_num.drop(values,axis=1,inplace=True)

In [89]:
# Visualizando o dataset para predição sem dados categóricos
display(df_num)

# Salvando o dataset para predição sem dados categóricos
df_num.to_csv('numeric-test.csv',index=False)

,MSSubClass,LotFrontage,LotArea,Street,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,...,GarageQual,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20,80.0,11622,1,3,0.0,2,5,6,1961,...,3.0,140,0,0,0,120,0,0,6,2010
1,20,81.0,14267,1,2,0.0,2,6,6,1958,...,3.0,393,36,0,0,0,0,12500,6,2010
2,60,74.0,13830,1,2,0.0,2,5,5,1997,...,3.0,212,34,0,0,0,0,0,3,2010
3,60,78.0,9978,1,2,0.0,2,6,6,1998,...,3.0,360,36,0,0,0,0,0,6,2010
4,120,43.0,5005,1,2,0.0,2,8,5,1992,...,3.0,0,82,0,0,144,0,0,1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,1,3,0.0,2,4,7,1970,...,0.0,0,0,0,0,0,0,0,6,2006
1455,160,21.0,1894,1,3,0.0,2,4,5,1970,...,3.0,0,24,0,0,0,0,0,4,2006
1456,20,160.0,20000,1,3,0.0,2,5,7,1960,...,3.0,474,0,0,0,0,0,0,9,2006
1457,85,62.0,10441,1,3,0.0,2,5,5,1992,...,0.0,80,32,0,0,0,0,700,7,2006


## Conclusão

Nesta parte realizamos a visualização dos dados, e entendmos como eles estão distribuidos, além disso sinalizamos os atributos que serão removidos de imediato e também foi explicado sucintamente como o pré-processamento do conjunto de dados iria ser realizado, além obviamente de chegar algumas propriedades dos dados